# BERT2BERT model

In [ ]:
# Install necessary libraries
!pip install transformers datasets rouge-score -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, EncoderDecoderModel, AdamW
from torch.utils.data import Dataset, DataLoader
import torch
from rouge_score import rouge_scorer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Dataset as df

In [ ]:
# df = pd.read_json("/content/drive/MyDrive/Binus/Sem 5/textmining/train.jsonl", lines=True)
# dfval = pd.read_json("/content/drive/MyDrive/Binus/Sem 5/textmining/dev.jsonl", lines=True)
# dftest = pd.read_json("/content/drive/MyDrive/Binus/Sem 5/textmining/test.jsonl", lines=True)

df = pd.read_json("/content/drive/MyDrive/sem 5/train.jsonl", lines=True)
dfval = pd.read_json("/content/drive/MyDrive/sem 5/dev.jsonl", lines=True)
dftest = pd.read_json("/content/drive/MyDrive/sem 5/test.jsonl", lines=True)

In [ ]:
df.head()

,source,source_labels,rouge_scores,paper_id,target,ic,title
0,[Due to the success of deep learning to solvin...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.30188678746885, 0.37209301838831804, 0.6037...",SysEexbRb,[We provide necessary and sufficient analytica...,True,Critical Points of Linear Neural Networks: Ana...
1,[The backpropagation (BP) algorithm is often t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.130434779206049, 0.14285713922902...",SygvZ209F7,"[Biologically plausible learning algorithms, p...",True,Biologically-Plausible Learning Algorithms Can...
2,"[We introduce the 2-simplicial Transformer, an...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.333333328395061, 0.8888888839111111, 0.1142...",rkecJ6VFvr,[We introduce the 2-simplicial Transformer and...,True,Logic and the 2-Simplicial Transformer
3,"[We present Tensor-Train RNN (TT-RNN), a novel...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.066666662222222, 0.06451612466181, 0.060606...",HJJ0w--0W,[Accurate forecasting over very long time hori...,True,Long-term Forecasting using Tensor-Train RNNs
4,[Recent efforts on combining deep models with ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.27777777279320903, 0.571428566658163, 0.095...",HyH9lbZAW,[We propose a variational message-passing algo...,True,Variational Message Passing with Structured In...


# Data Preparation

In [ ]:
dfval.head()

,source,source_labels,rouge_scores,paper_id,target,ic,title
0,[Mixed precision training (MPT) is becoming a ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.23999999580000003, 0.260869560822306, 0.199...",rJlnfaNYvB,[We devise adaptive loss scaling to improve mi...,True,Adaptive Loss Scaling for Mixed Precision Trai...
1,"[Many real-world problems, e.g. object detecti...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.054054049086925, 0.29268292183224204, 0.974...",rJVoEiCqKQ,[We present a novel approach for learning to p...,True,Deep Perm-Set Net: Learn to predict sets with ...
2,[Foveation is an important part of human visio...,"[0, 0, 1, 0, 0]","[0.11764705382352901, 0.11764705382352901, 0.3...",rkldVXKU8H,[We compare object recognition performance on ...,False,Foveated Downsampling Techniques
3,[We explore the concept of co-design in the co...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.12499999548828102, 0.488888883911111, 0.204...",BJfIVjAcKm,[We develop methods to train deep neural model...,True,Training for Faster Adversarial Robustness Ver...
4,[Batch Normalization (BatchNorm) has shown to ...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.1999999952, 0.239999995008, 0.3999999950080...",BJlEEaEFDS,[Investigation of how BatchNorm causes adversa...,True,Towards an Adversarially Robust Normalization ...


In [ ]:
dftest.head()

,source,source_labels,rouge_scores,paper_id,target,ic,title
0,[Incremental class learning involves sequentia...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.28571428104489804, 0.18181817681818102, 0.2...",SJ1Xmf-Rb,"[FearNet is a memory efficient neural-network,...",True,FearNet: Brain-Inspired Model for Incremental ...
1,[Multi-view learning can provide self-supervis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0.19999999580000002, 0.0, 0.15789473418282501...",S1xzyhR9Y7,[Multi-view learning improves unsupervised sen...,True,Improving Sentence Representations with Multi-...
2,[We show how discrete objects can be learnt in...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.9787233992575821, 0.33333332860555503, 0.41...",HJDUjKeA-,[We show how discrete objects can be learnt in...,True,Learning objects from pixels
3,[Most recent gains in visual recognition have ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.11764705384083, 0.146341458655562, 0.199999...",BJgLg3R9KQ,[A large-scale dataset for training attention ...,True,Learning what and where to attend
4,"[In recent years, deep neural networks have de...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.05882352484429101, 0.270270265887509, ...",BklpOo09tQ,[We proposed a time-efficient defense method a...,True,EFFICIENT TWO-STEP ADVERSARIAL DEFENSE FOR DEE...


In [ ]:
# drop source_labels, rouge_scores, paper_id, ic and title from df
df = df.drop(['source_labels', 'rouge_scores', 'paper_id', 'ic', 'title'], axis=1)
dfval = dfval.drop(['source_labels', 'rouge_scores', 'paper_id', 'ic', 'title'], axis=1)
dftest = dftest.drop(['source_labels', 'rouge_scores', 'paper_id', 'ic', 'title'], axis=1)
df.head()

,source,target
0,[Due to the success of deep learning to solvin...,[We provide necessary and sufficient analytica...
1,[The backpropagation (BP) algorithm is often t...,"[Biologically plausible learning algorithms, p..."
2,"[We introduce the 2-simplicial Transformer, an...",[We introduce the 2-simplicial Transformer and...
3,"[We present Tensor-Train RNN (TT-RNN), a novel...",[Accurate forecasting over very long time hori...
4,[Recent efforts on combining deep models with ...,[We propose a variational message-passing algo...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1992 entries, 0 to 1991
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  1992 non-null   object
 1   target  1992 non-null   object
dtypes: object(2)
memory usage: 31.2+ KB


In [ ]:
dfval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619 entries, 0 to 618
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  619 non-null    object
 1   target  619 non-null    object
dtypes: object(2)
memory usage: 9.8+ KB


In [ ]:
dftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  618 non-null    object
 1   target  618 non-null    object
dtypes: object(2)
memory usage: 9.8+ KB


# Text Cleansing

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def cleansing(df):
    df_clean=df.astype(str).str.lower()
    df_clean=[re.sub(r"\d+","",i )for i in df_clean]
    df_clean=[re.sub(r'[^\w]', ' ', i)for i in df_clean]
    df_clean=[re.sub(r'\s+',' ',i)for i in df_clean]
    return df_clean

In [ ]:
df['clean_text']=cleansing(df['source'])
df['clean_target']=cleansing(df['target'])
dfval['clean_text']=cleansing(dfval['source'])
dfval['clean_target']=cleansing(dfval['target'])
dftest['clean_text']=cleansing(dftest['source'])
dftest['clean_target']=cleansing(dftest['target'])

df = df.drop(['source','target'], axis=1)
dfval = dfval.drop(['source','target'], axis=1)
dftest = dftest.drop(['source','target'], axis=1)

In [ ]:
df.rename(columns={"clean_text": "text", "clean_target": "target"}, inplace=True)
dfval.rename(columns={"clean_text": "text", "clean_target": "target"}, inplace=True)
dftest.rename(columns={"clean_text": "text", "clean_target": "target"}, inplace=True)

In [ ]:
df.head()

,text,target
0,due to the success of deep learning to solvin...,we provide necessary and sufficient analytica...
1,the backpropagation bp algorithm is often tho...,biologically plausible learning algorithms pa...
2,we introduce the simplicial transformer an ex...,we introduce the simplicial transformer and s...
3,we present tensor train rnn tt rnn a novel fa...,accurate forecasting over very long time hori...
4,recent efforts on combining deep models with ...,we propose a variational message passing algo...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1992 entries, 0 to 1991
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1992 non-null   object
 1   target  1992 non-null   object
dtypes: object(2)
memory usage: 31.2+ KB


In [ ]:
dfval.head()

,text,target
0,mixed precision training mpt is becoming a pr...,we devise adaptive loss scaling to improve mi...
1,many real world problems e g object detection...,we present a novel approach for learning to p...
2,foveation is an important part of human visio...,we compare object recognition performance on ...
3,we explore the concept of co design in the co...,we develop methods to train deep neural model...
4,batch normalization batchnorm has shown to be...,investigation of how batchnorm causes adversa...


In [ ]:
dfval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619 entries, 0 to 618
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    619 non-null    object
 1   target  619 non-null    object
dtypes: object(2)
memory usage: 9.8+ KB


In [ ]:
dftest.head()

,text,target
0,incremental class learning involves sequentia...,fearnet is a memory efficient neural network ...
1,multi view learning can provide self supervis...,multi view learning improves unsupervised sen...
2,we show how discrete objects can be learnt in...,we show how discrete objects can be learnt in...
3,most recent gains in visual recognition have ...,a large scale dataset for training attention ...
4,in recent years deep neural networks have dem...,we proposed a time efficient defense method a...


In [ ]:
dftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    618 non-null    object
 1   target  618 non-null    object
dtypes: object(2)
memory usage: 9.8+ KB


# Train, Validation, Test Spliting

In [ ]:
train_df = df
val_df = dfval
test_df = dftest

print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

Train: 1992, Validation: 619, Test: 618


# Convert Dataset to HuggingFace

In [ ]:
# class SummarizationDataset(Dataset): # class Cleansing yang final
#     def __init__(self, dataframe, tokenizer, max_input_len=512, max_output_len=128):
#         self.data = dataframe
#         self.tokenizer = tokenizer
#         self.max_input_len = max_input_len
#         self.max_output_len = max_output_len

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         row = self.data.iloc[idx]
#         #source = row["text"]
#         source = "summarize: " + " ".join(row["text"])
#         target = row["target"]

#         source_encodings = self.tokenizer(
#             source,
#             max_length=self.max_input_len,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt",
#         )
#         target_encodings = self.tokenizer(
#             target,
#             max_length=self.max_output_len,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt",
#         )

#         return {
#             "input_ids": source_encodings["input_ids"].squeeze(),
#             "attention_mask": source_encodings["attention_mask"].squeeze(),
#             "labels": target_encodings["input_ids"].squeeze(),
#         }

In [ ]:
# from torch.utils.data import Dataset, DataLoader

# class SCITldrDataset(Dataset): # class buat NonCleansing yang final
#     def __init__(self, dataset, tokenizer, max_input_len=512, max_output_len=128):
#         self.dataset = dataset  # Save the dataset object directly
#         self.tokenizer = tokenizer
#         self.max_input_len = max_input_len
#         self.max_output_len = max_output_len

#     def __len__(self):
#         return len(self.dataset)  # Use the length of the dataset

#     def __getitem__(self, idx):
#         # Access the data entry using .iloc to ensure integer-based indexing
#         item = self.dataset.iloc[idx]
#         # Join the list of strings in 'source' into a single string
#         input_text = "summarize: " + " ".join(item["source"])
#         target_text = item["target"]

#         # Tokenize input and output
#         inputs = self.tokenizer(
#             input_text,
#             max_length=self.max_input_len,
#             truncation=True,
#             padding="max_length",
#             return_tensors="pt",
#         )
#         targets = self.tokenizer(
#             target_text,
#             max_length=self.max_output_len,
#             truncation=True,
#             padding="max_length",
#             return_tensors="pt",
#         )

#         return {
#             "input_ids": inputs["input_ids"].squeeze(),
#             "attention_mask": inputs["attention_mask"].squeeze(),
#             "labels": targets["input_ids"].squeeze(),
#         }

In [ ]:
class SummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_input_len=128, max_output_len=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_output_len = max_output_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        # source = row["clean_text"]
        source = "summarize: " + row["text"] # correct prefix
        target = row["target"]

        source_encodings = self.tokenizer(
            source,
            max_length=self.max_input_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        target_encodings = self.tokenizer(
            target,
            max_length=self.max_output_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": source_encodings["input_ids"].squeeze(),
            "attention_mask": source_encodings["attention_mask"].squeeze(),
            "labels": target_encodings["input_ids"].squeeze(),
        }

# Set Model

In [ ]:
model_name = "bert2bert"  # Using a BERT-to-BERT EncoderDecoder model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model moved to {device}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "return_dict": false,
  "transformers_version": "4.47.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "bert-base-uncased",
  "add_cross_attention"

Model moved to cuda


In [ ]:
# Prepare datasets
train_dataset = SummarizationDataset(train_df, tokenizer) # pake yang cleansing karena nama collumnnya beda
val_dataset = SummarizationDataset(val_df, tokenizer)
test_dataset = SummarizationDataset(test_df, tokenizer)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

# Model Training Arc

In [ ]:
# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

epochs = 10  # Maximum epochs
patience = 3  # Early stopping patience
best_val_loss = float("inf")
patience_counter = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"Epoch {epoch + 1}/{epochs}")

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Print progress
        if (step + 1) % 10 == 0:
            print(f"Step {step + 1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1} Training Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch + 1} Validation Loss: {avg_val_loss:.4f}")

    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pt")
        print("Best model saved.")
    else:
        patience_counter += 1
        print(f"No improvement. Patience: {patience_counter}/{patience}")

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:649: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the deco

Step 10/249, Loss: 4.3000
Step 20/249, Loss: 2.0002
Step 30/249, Loss: 1.4845
Step 40/249, Loss: 1.0588
Step 50/249, Loss: 1.0291
Step 60/249, Loss: 0.4639
Step 70/249, Loss: 0.7273
Step 80/249, Loss: 0.7514
Step 90/249, Loss: 0.9598
Step 100/249, Loss: 0.7461
Step 110/249, Loss: 0.8104
Step 120/249, Loss: 0.6681
Step 130/249, Loss: 0.6408
Step 140/249, Loss: 0.6672
Step 150/249, Loss: 0.8291
Step 160/249, Loss: 0.5806
Step 170/249, Loss: 0.7391
Step 180/249, Loss: 0.7406
Step 190/249, Loss: 0.8947
Step 200/249, Loss: 0.8613
Step 210/249, Loss: 0.6466
Step 220/249, Loss: 0.7393
Step 230/249, Loss: 0.7381
Step 240/249, Loss: 0.7149
Epoch 1 Training Loss: 1.1607
Epoch 1 Validation Loss: 2.0244
Best model saved.
Epoch 2/10
Step 10/249, Loss: 0.5087
Step 20/249, Loss: 0.4747
Step 30/249, Loss: 0.4941
Step 40/249, Loss: 0.6409
Step 50/249, Loss: 0.7503
Step 60/249, Loss: 0.6333
Step 70/249, Loss: 0.6380
Step 80/249, Loss: 0.4229
Step 90/249, Loss: 0.7055
Step 100/249, Loss: 0.5384
Step 110/

# Model Tournament Arc

In [ ]:
# Load the best model
model.load_state_dict(torch.load("best_model.pt"))
model.eval()

predictions = []
references = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150)
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_refs = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

        predictions.extend(decoded_preds)
        references.extend(decoded_refs)

# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

rouge1, rouge2, rougeL = 0, 0, 0
for ref, pred in zip(references, predictions):
    scores = scorer.score(ref, pred)
    rouge1 += scores["rouge1"].fmeasure
    rouge2 += scores["rouge2"].fmeasure
    rougeL += scores["rougeL"].fmeasure

n = len(predictions)
print(f"ROUGE-1: {rouge1 / n:.4f}")
print(f"ROUGE-2: {rouge2 / n:.4f}")
print(f"ROUGE-L: {rougeL / n:.4f}")


<ipython-input-29-5a2e5aa9e4ab>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pt"))
We strongly recommend passing in an `atte

ROUGE-1: 0.3868
ROUGE-2: 0.1215
ROUGE-L: 0.2435


In [ ]:
print(f"ROUGE-1: {rouge1 * 100 / n:.4f} %")
print(f"ROUGE-2: {rouge2 * 100 / n:.4f} %")
print(f"ROUGE-L: {rougeL * 100 / n:.4f} %")

ROUGE-1: 38.6842 %
ROUGE-2: 12.1461 %
ROUGE-L: 24.3546 %
